In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from timeit import default_timer as timer

In [ ]:
def compute_Q(A):
  n=len(A) # number of data points
  Q_matrix=np.matmul(A.T,A)/n
  return Q_matrix

In [ ]:
iteration_num=0
tolerance=1
matrix_decomposition=2
power_method=3

In [ ]:
def compute_principal_direc_power(A, tol=1e-6, max_iter=1000):
  n = A.shape[0]
  x = np.ones(n) / np.sqrt(n)
  delta = 1

  for i in range(max_iter):
      x_new = A @ x
      x_new = x_new / np.linalg.norm(x_new)
      delta = np.linalg.norm(x_new - x)
      x = x_new

      if delta < tol:
          break

  return x

In [ ]:
def compute_cosine(a,b):
  assert type(a) is np.ndarray and type(b) is np.ndarray
  return (np.dot(a,b))/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
def compute_thres(score):
  a=np.array(score)
  return np.mean(a)+2*np.std(a)

In [ ]:
# let's define a seed value
seed=17


In [ ]:
def guass_2d(mu,std,size):
  data=[]
  np.random.seed(seed)
  for i in range(size):
    x=np.random.normal(mu,std)
    y=np.random.normal(mu,std)
    data.append(x)
    data.append(y)
  return pd.DataFrame(np.array(data).reshape(size,2))

In [ ]:

data_1=guass_2d(0,1,10000)
data_2=guass_2d(0,15,100)
data_large=pd.concat([data_1,data_2])
data_large

,0,1
0,0.276266,-1.854628
1,0.623901,1.145311
2,1.037190,1.886639
3,-0.111698,-0.362101
4,0.148675,-0.437783
...,...,...
95,5.074319,7.654283
96,0.756203,-12.184699
97,-2.201941,30.784561
98,17.180586,1.147199


In [ ]:

def dchange(x):
  return np.array(list(x))

$\large \text{PCAOD: Principal Component Analysis Otlier Detection without Oversampling with LOO(leave one out) and with or without strategy}$

In [ ]:
def PCAOD(data):
  start=timer()
  outlier_score=[]
  data_score={}
  Q_mat_init=compute_Q(np.array(data))
  init_dir=compute_principal_direc_power(Q_mat_init)
  for i in range(len(data)):

    data_rem=data.drop(i)

    Q_mat=compute_Q(np.array(data_rem))
    dir=compute_principal_direc_power(Q_mat)

    cosine_val=compute_cosine(init_dir,dir)

    outlier_scr=1-abs(cosine_val)

    outlier_score.append(outlier_scr)
    data_score[tuple(data.iloc[i])]=outlier_scr

  end=timer()

  return outlier_score,end-start





$\large \text{Over-sampling principal Component Analysis outlier detection(used oversampling technique)}$

In [ ]:
def OPCAOD(dat,r=0.1):
  start=timer()
  n=len(dat) # number of data points
  dim=dat.shape[1]
  #data_score={}
  Q_mat_init=compute_Q(np.array(dat))
  mu_init=np.array(dat.mean())
  prin_dir_init=compute_principal_direc_power(Q_mat_init)
  #prin_dir_list=[]
  #cosine_sim_list=[]
  outlier_score_list=[]
  m=(1+r)
  for i in range(len(dat)):
    mu=((mu_init)+r*np.array(dat.iloc[i]))/m

    data_point=np.array(dat.iloc[i])
    cov_mat=(Q_mat_init/m)+((r/m)*np.outer(data_point,data_point))-np.outer(mu,mu)
    # finding principal direction
    prin_dir=compute_principal_direc_power(cov_mat)
    #prin_dir_list.append(prin_dir)
    # finding cosine similiraty (cos theta)
    cosine_sim=compute_cosine( prin_dir_init,prin_dir)
    #cosine_sim_list.append(cosine_sim)
    # h
    outlier_score=(1-abs(cosine_sim))
    outlier_score_list.append(outlier_score)
    #data_score[tuple(dat.iloc[i])]=outlier_score
  Sc=np.array(outlier_score_list)
  mean=Sc.mean()
  outlier_score_list=abs(Sc-mean)

  end=timer()

  return outlier_score_list,end-start

$\large \text{LOF: Local Outlier Factor using k nearest neighbours to find density around data point}$

In [ ]:
def dchange(x):
  return np.array(list(x))



def LOF(data,k=100):
  start_time=timer() # starting time
  LRDs={}
  neigh_point={}
  for i in range(len(data)):
    data_point=np.array(data.iloc[i]) # target data point

    data_upd=data.drop(i)
    dist_list=[]
    for j in range(len(data_upd)):
      neigh=np.array(data_upd.iloc[j])
      # distance between target instance and neighbourhood
      dist=np.linalg.norm(data_point-neigh)
      # distances and nieghbourhood
      dist_list.append((dist,list(neigh)))
    # sorting the distance matrix in ascending order
    dist_list.sort()
    dist_list=dist_list[:k]
    # finding the LRD score
    sum=0
    neighbours=[]
    for i in dist_list:
      sum+=i[0]
      neighbours.append(i[1])
    # finding LRD score which is reciprocal of avg distance
    LRD=k/sum
    LRDs[tuple(data_point)]=LRD
    neigh_point[tuple(data_point)]=neighbours

  LRD_score={}
  for i in range(len(data)):
    data_inst=list(data.iloc[i])
    LRD_own=LRDs[tuple(data_inst)]
    sum=0
    for j in neigh_point[tuple(data_inst)]:
      sum+=LRDs[tuple(j)]
    avg=sum/k # avg of LRDs of neighbours

    ratio=avg/LRD_own # this is LRD score

    LRD_score[tuple(data_inst)]=ratio
  score=list(LRD_score.values())

  # ending time
  end_time=timer()
  return score,end_time-start_time


$\large \text{fast_ABOD: approximate ABOF(Angle Based outlier factor) here approximation done by taking k nearest neighbours}$

In [ ]:
def fast_ABOD(data,k=40):
  start_time=timer() # starting time
  ABOF_score=[]
  data_inst=[]
  for i in range(len(data)):
    data_1_point=np.array(data.iloc[i]) # target data point
    data_inst.append(data_1_point)
    data_1_upd=data.drop(i)
    dist_list=[]
    for j in range(len(data_1_upd)):
      neigh=np.array(data_1_upd.iloc[j])
      # distance between target instance and neighbourhood
      dist=np.linalg.norm(data_1_point-neigh)
      # distances and nieghbourhood
      dist_list.append((dist,list(neigh)))
    # sorting the distance matrix in ascending order
    dist_list.sort()
    dist_list=dist_list[:k]

    # finding FastABOD score(approximate ABOF score)

    angles=[]
    for l in range(k):
      ang_num=np.dot(data_1_point,np.array(dist_list[l][1]))
      ang_den=np.linalg.norm(data_1_point)*np.linalg.norm(np.array(dist_list[l][1]))
      angles.append(ang_num/ang_den)
    score=np.var(np.array(angles))

    # appending the score
    ABOF_score.append(score)


  end_time=timer()
  return ABOF_score,end_time-start_time



$ \text{Data is synthetic data with 200 data points generated by 2d guassian with mean 0 and std 1 and 10 data points with mean 0 and std 15. }$

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
data_normal=guass_2d(0,1,200)
data_deviate=guass_2d(0,15,10)
data=pd.concat([data_normal,data_deviate])
data=data.reset_index(drop=True)

In [ ]:
data['label']=[0 for i in range(200)]+[1 for i in range(10)]
data_X=data.drop('label',axis=1)
data_X

,0,1
0,0.276266,-1.854628
1,0.623901,1.145311
2,1.037190,1.886639
3,-0.111698,-0.362101
4,0.148675,-0.437783
...,...,...
205,32.568855,17.284654
206,-27.282185,-2.070740
207,8.097594,-26.629234
208,19.723148,-7.101721


In [ ]:
data

,0,1,label
0,0.276266,-1.854628,0
1,0.623901,1.145311,0
2,1.037190,1.886639,0
3,-0.111698,-0.362101,0
4,0.148675,-0.437783,0
...,...,...,...
205,32.568855,17.284654,1
206,-27.282185,-2.070740,1
207,8.097594,-26.629234,1
208,19.723148,-7.101721,1


In [ ]:
print('data_X:Synthetic data_X ')
print('\n*****************************************************************\n')
print('LOCAL OUTLIER FACTOR')
score_LOF,time_LOF=LOF(data_X)
print('time taken by LOF method:',time_LOF)
print('roc_auc_score is:',roc_auc_score(data['label'],score_LOF))

print("\n**********************************************************\n")
print('PRINCIPAL COMPONENT ANALYSIS OUTLIER FACTOR:')
score_PCAOD,time_PCAOD=PCAOD(data_X)
print('roc_auc_score is:',roc_auc_score(data['label'],score_PCAOD))
print('time taken by PCAOD method:',time_PCAOD)
print("\n**********************************************************\n")
print('OVERSAMPLING PRINCIPAL COMPONENT ANALYSIS OUTLIER FACTOR ')
score_OPCAOD,time_OPCAOD=OPCAOD(data_X,r=0.1)
print('time taken by OPCAOD method:',time_OPCAOD)
print('roc_auc_score is:',roc_auc_score(data['label'],score_OPCAOD))

print("\n**********************************************************\n")
print('Fast ABOD: Approximate Angle based outlier factor:')
score_fastABOD,time_fastABOD=fast_ABOD(data_X,k=40)
print('time taken by fastABOD method:',time_fastABOD)
print('roc_auc_score is:',roc_auc_score(data['label'],score_fastABOD))
print("\n**********************************************************\n")





data_X:Synthetic data_X 

*****************************************************************

LOCAL OUTLIER FACTOR
time taken by LOF method: 2.865635088999994
roc_auc_score is: 1.0

**********************************************************

PRINCIPAL COMPONENT ANALYSIS OUTLIER FACTOR:
roc_auc_score is: 1.0
time taken by PCAOD method: 0.2365005910000093

**********************************************************

OVERSAMPLING PRINCIPAL COMPONENT ANALYSIS OUTLIER FACTOR 
time taken by OPCAOD method: 0.07275643599999171
roc_auc_score is: 0.9

**********************************************************

Fast ABOD: Approximate Angle based outlier factor:
time taken by fastABOD method: 2.2004789730000027
roc_auc_score is: 0.637

**********************************************************



In [ ]:
pen_data=pd.read_csv('pendigits_train.csv',header=None)
pen_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [ ]:
pen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7494 entries, 0 to 7493
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       7494 non-null   int64
 1   1       7494 non-null   int64
 2   2       7494 non-null   int64
 3   3       7494 non-null   int64
 4   4       7494 non-null   int64
 5   5       7494 non-null   int64
 6   6       7494 non-null   int64
 7   7       7494 non-null   int64
 8   8       7494 non-null   int64
 9   9       7494 non-null   int64
 10  10      7494 non-null   int64
 11  11      7494 non-null   int64
 12  12      7494 non-null   int64
 13  13      7494 non-null   int64
 14  14      7494 non-null   int64
 15  15      7494 non-null   int64
 16  16      7494 non-null   int64
dtypes: int64(17)
memory usage: 995.4 KB


In [ ]:
data_X=pen_data[pen_data.columns[:16]]

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data_X=scaler.fit_transform(data_X)

data_X=pd.DataFrame(data_X)
pen_data_std=pd.concat([data_X,pen_data[16]],axis=1)
pen_data_std


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.288588,0.909384,-0.495371,-0.096221,0.175282,-1.024435,-0.830296,-1.454775,-1.696354,-0.403127,-0.145910,0.626242,2.004589,1.631456,-0.164071,1.939043,8
1,-1.121985,0.256460,-0.495371,0.871327,-0.254211,0.363667,-0.732506,0.013058,-1.250961,-0.694479,-0.657650,-1.301451,0.632802,-1.017440,1.280702,-0.630165,2
2,-1.121985,-1.642955,-0.343015,-0.758227,0.604775,0.911602,1.581854,1.807076,0.560306,1.490664,-0.307512,0.553499,-1.181496,-0.325115,-0.741980,-0.797722,1
3,-1.121985,0.909384,-1.257152,0.463939,-1.313628,0.107964,-1.058472,0.013058,0.857235,-0.002517,1.039170,0.335270,0.854058,-0.385317,0.486077,-0.797722,4
4,-1.121985,-1.049388,0.342588,0.005626,1.406496,1.276892,0.962518,1.154706,0.085219,0.944378,-0.576849,0.153412,-0.960240,-0.475620,0.004486,-0.797722,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7489,-1.121985,-0.159037,-1.180974,-1.216539,0.146649,-1.134022,-0.341347,-1.454775,-1.399425,-0.148194,-1.573393,1.135444,-0.561979,1.812062,1.280702,1.994895,5
7490,0.348612,0.909384,-1.523775,-0.656380,-0.769603,-0.330384,1.581854,0.665428,0.857235,1.854854,-0.469114,1.499160,-1.491254,0.066199,-0.982776,-0.797722,4
7491,1.879234,0.790670,0.761567,0.871327,-0.769603,0.802015,-1.580018,0.437099,-0.657103,0.616606,-0.092043,-0.355790,-0.827486,-1.077642,-1.127253,-0.658091,5
7492,0.648734,-1.168101,1.942327,0.871327,0.948370,1.130776,0.669148,0.176150,-0.182017,-0.949413,-1.654194,-1.301451,-0.429226,-1.047541,1.280702,-0.797722,1


In [ ]:
# Separating data points based on their labels

pen_data_std_0=pen_data_std[pen_data_std[16]==0]
pen_data_std_1=pen_data_std[pen_data_std[16]==1]
pen_data_std_2=pen_data_std[pen_data_std[16]==2]
pen_data_std_3=pen_data_std[pen_data_std[16]==3]
pen_data_std_4=pen_data_std[pen_data_std[16]==4]
pen_data_std_5=pen_data_std[pen_data_std[16]==5]
pen_data_std_6=pen_data_std[pen_data_std[16]==6]
pen_data_std_7=pen_data_std[pen_data_std[16]==7]
pen_data_std_8=pen_data_std[pen_data_std[16]==8]
pen_data_std_9=pen_data_std[pen_data_std[16]==9]


In [ ]:
#pen_data_std_0

$\large \text{PENDIGITS DATA:}$

In [ ]:
data=pd.concat([pen_data_std_0,pen_data_std_1.iloc[:20]]).reset_index(drop=True)
data_X=data[data.columns[:16]]
data_X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1.121985,-2.711375,-1.447597,-1.063769,-1.141831,-2.193363,0.375779,-1.454775,1.272935,0.325254,0.742900,2.299335,-0.827486,1.932466,-1.127253,0.794070
1,0.588710,0.909384,-0.685816,-0.554533,-1.456793,-1.243609,-0.862893,-1.454775,0.530613,-0.767317,1.039170,0.517128,0.898309,1.541152,-0.501185,1.576003
2,-0.041546,0.434530,-1.257152,0.005626,-1.456793,-1.024435,-0.830296,-1.454775,0.589999,-1.022251,1.039170,0.371641,1.296570,1.601354,-0.260389,1.994895
3,-0.311656,-0.515177,-1.485686,-2.082240,-0.998667,-2.376008,0.603955,-1.356920,1.272935,0.215997,0.823701,1.753761,-0.473477,1.932466,-1.127253,1.240889
4,0.558698,0.850027,-0.838172,-0.554533,-1.456793,-1.499312,-0.504330,-1.454775,0.827542,-0.403127,1.039170,1.208187,0.146039,1.932466,-0.669742,1.268815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1.519087,-0.277750,-1.066707,-0.147144,1.406496,1.276892,1.581854,1.513510,0.233684,1.235730,-0.873119,0.371641,-2.155021,-0.385317,-1.127253,-0.797722
796,-1.121985,0.909384,1.256725,0.718556,1.263332,0.546312,1.581854,0.632810,1.124471,0.507349,0.554364,-0.173932,0.411547,-0.626126,-0.573424,-0.797722
797,0.048490,-1.346171,1.447170,0.260244,0.805205,1.276892,0.180199,0.632810,-0.568024,-0.148194,-1.654194,-1.301451,-0.915989,-0.957238,1.280702,-0.797722
798,-1.121985,-1.168101,0.114054,-0.350839,1.206066,1.130776,1.581854,1.807076,0.411841,1.490664,-0.361379,0.517128,-1.269998,-0.325115,-1.006855,-0.797722


In [ ]:
print('\n********************************************\n')
data_list=[pen_data_std_1,pen_data_std_2,pen_data_std_3,pen_data_std_4,pen_data_std_5,pen_data_std_6,pen_data_std_7,pen_data_std_8,pen_data_std_9]
print('taking 0 class as normal instances and 1 class as outliers(20 data points)')
# time taken by various methods
time_LOF_list=[]
time_PCAOD_list=[]
time_OPCAOD_list=[]
time_fastABOD_list=[]
# auc scores
auc_score_LOF_list=[]
auc_score_PCAOD_list=[]
auc_score_OPCAODlist=[]
auc_score_fastABOD_list=[]
seeds=[2,12,22,32,42,52]
# taking data
k=0
for i in data_list:
  k+=1
  print(k)
  data=pd.concat([pen_data_std_0,i.iloc[:20]]).reset_index(drop=True)
  data_X=data[data.columns[:16]]
  #print('OPCAOD METHOD:')
  score_OPCAOD,time_OPCAOD=OPCAOD(data_X)
  roc_auc_score_OPCAOD=roc_auc_score(data[16],score_OPCAOD)
  print('roc_auc score is :',roc_auc_score(data[16],score_OPCAOD))
  print('time taken:',time_OPCAOD)
  time_OPCAOD_list.append(time_OPCAOD)
  auc_score_OPCAODlist.append(roc_auc_score_OPCAOD)



********************************************

taking 0 class as normal instances and 1 class as outliers(20 data points)
1
roc_auc score is : 0.9898076923076923
time taken: 0.20638705700002902
2
roc_auc score is : 0.9996794871794872
time taken: 0.23101335400002654
3
roc_auc score is : 0.9773076923076923
time taken: 0.2121534259999862
4
roc_auc score is : 0.9037820512820512
time taken: 0.17079849100002775
5
roc_auc score is : 0.7180769230769231
time taken: 0.21348731599999837
6
roc_auc score is : 0.9773717948717949
time taken: 0.17445905099998527
7
roc_auc score is : 0.9966025641025641
time taken: 0.20008668700000953
8
roc_auc score is : 0.889102564102564
time taken: 0.19185224799997513
9
roc_auc score is : 0.9503205128205128
time taken: 0.21279160599999614


In [ ]:
print('\n********************************************\n')
data_list=[pen_data_std_1,pen_data_std_2,pen_data_std_3,pen_data_std_4,pen_data_std_5,pen_data_std_6,pen_data_std_7,pen_data_std_8,pen_data_std_9]
print('taking 0 class as normal instances and 1 class as outliers(20 data points)')
# time taken by various methods
time_LOF_list=[]
time_PCAOD_list=[]
time_OPCAOD_list=[]
time_fastABOD_list=[]
# auc scores
auc_score_LOF_list=[]
auc_score_PCAOD_list=[]
auc_score_OPCAODlist=[]
auc_score_fastABOD_list=[]
seeds=[2,12,22,32,42,52]
# taking data
k=0
for i in data_list:
  k+=1
  print(k)
  data=pd.concat([pen_data_std_0,i.iloc[:20]]).reset_index(drop=True)
  data_X=data[data.columns[:16]]

  score_fastABOD,time_fastABOD=fast_ABOD(data_X)
  #print('fast ABOD method ************')
  roc_auc_score_fastABOD=roc_auc_score(data[16],score_fastABOD)
  #print('roc_auc score is :',roc_auc_score(data[16],score_fastABOD))
  #print('time taken:',time_fastABOD)
  time_fastABOD_list.append(time_fastABOD)
  auc_score_fastABOD_list.append(roc_auc_score_fastABOD)
  #print('OPCAOD METHOD:')
  score_OPCAOD,time_OPCAOD=OPCAOD(data_X)
  roc_auc_score_OPCAOD=roc_auc_score(data[16],score_OPCAOD)
  #print('roc_auc score is :',roc_auc_score(data[16],score_OPCAOD))
  #print('time taken:',time_OPCAOD)
  time_OPCAOD_list.append(time_OPCAOD)
  auc_score_OPCAODlist.append(roc_auc_score_OPCAOD)
  #print('LOF METHOD')
  score_LOF,time_LOF=LOF(data_X)

  roc_auc_score_LOF=roc_auc_score(data[16],score_LOF)
  #print('roc_auc score is :',roc_auc_score(data[16],score_LOF))
  #print('time taken:',time_LOF)
  time_LOF_list.append(time_LOF)
  auc_score_LOF_list.append(roc_auc_score_LOF)

  #print('PCAOD method **************************************')
  score_PCAOD,time_PCAOD=PCAOD(data)

  #print('roc_auc score is :',roc_auc_score(data[16],score_PCAOD))
  #print('time taken:',time_PCAOD)
  roc_auc_score_PCAOD=roc_auc_score(data[16],score_PCAOD)
  #print('roc_auc score is :',roc_auc_score(data[16],score_PCAOD))
  #print('time taken:',time_PCAOD)
  time_PCAOD_list.append(time_PCAOD)
  auc_score_PCAOD_list.append(roc_auc_score_PCAOD)










********************************************

taking 0 class as normal instances and 1 class as outliers(20 data points)
1
2
3
4
5
6
7
8
9


In [ ]:
avg_time=[]
avg_time.append(np.mean(np.array(time_PCAOD_list)))
avg_time.append(np.mean(np.array(time_OPCAOD_list)))
avg_time.append(np.mean(np.array(time_fastABOD_list)))
avg_time.append(np.mean(np.array(time_LOF_list)))
avg_time_table=pd.DataFrame()
avg_time_table['method']=['PCAOD','OPCAOD','fastABOD','LOF']
avg_time_table['avg_time']=avg_time
avg_time_table

,method,avg_time
0,PCAOD,0.663560
1,OPCAOD,0.193596
2,fastABOD,32.449585
3,LOF,33.282286


|index|method|avg\_time|
|---|---|---|
|0|PCAOD|0\.6635599300000068|
|1|OPCAOD|0\.19359602977777335|
|2|fastABOD|32\.449584638777736|
|3|LOF|33\.28228576244444|

In [ ]:
table=pd.DataFrame()
table['Scenario']=[f'0 vs {i+1}' for i in range(9)]
table['LOF AUC_score']=auc_score_LOF_list
table['fastABOD AUC_score']=auc_score_fastABOD_list
table['PCAOD AUC_score']=auc_score_PCAOD_list
table['OPCAOD AUC_score']=auc_score_OPCAODlist
table

,Scenario,LOF AUC_score,fastABOD AUC_score,PCAOD AUC_score,OPCAOD AUC_score
0,0 vs 1,0.987949,0.999679,0.104167,0.989808
1,0 vs 2,0.983846,1.000000,0.242885,0.999679
2,0 vs 3,0.984679,1.000000,0.147372,0.977308
3,0 vs 4,0.980064,0.991282,0.182179,0.903782
4,0 vs 5,0.996346,0.999808,0.806090,0.718077
5,0 vs 6,0.956667,0.986090,0.252051,0.977372
6,0 vs 7,0.990192,0.999295,0.453269,0.996603
7,0 vs 8,0.989231,0.985192,0.696795,0.889103
8,0 vs 9,0.987756,0.997885,0.408013,0.950321


|index|Scenario|LOF AUC\_score|fastABOD AUC\_score|PCAOD AUC\_score|OPCAOD AUC\_score|
|---|---|---|---|---|---|
|0|0 vs 1|0\.9879487179487179|0\.9996794871794872|0\.10416666666666666|0\.9898076923076923|
|1|0 vs 2|0\.9838461538461537|1\.0|0\.24288461538461537|0\.9996794871794872|
|2|0 vs 3|0\.9846794871794873|1\.0|0\.14737179487179486|0\.9773076923076923|
|3|0 vs 4|0\.9800641025641026|0\.9912820512820513|0\.18217948717948718|0\.9037820512820512|
|4|0 vs 5|0\.9963461538461539|0\.9998076923076923|0\.8060897435897436|0\.7180769230769231|
|5|0 vs 6|0\.9566666666666667|0\.9860897435897437|0\.252051282051282|0\.9773717948717949|
|6|0 vs 7|0\.9901923076923076|0\.9992948717948719|0\.45326923076923076|0\.9966025641025641|
|7|0 vs 8|0\.9892307692307692|0\.9851923076923077|0\.6967948717948718|0\.889102564102564|
|8|0 vs 9|0\.9877564102564103|0\.9978846153846155|0\.40801282051282056|0\.9503205128205128|